In [1]:
import os

import SimpleITK as sitk
import cv2


import numpy as np
import os
import pandas as pd
import math
import json
import random

from matplotlib import pyplot as plt

In [2]:
def read_images(folder = "dataset",
                classes = [
                            "normal",
                            "fatty",
                            "cirrhosis"
                        ]):
    image_names = {}
    images = []
    # Get all image names in folders
    for cls in classes:
        image_names[cls] = os.listdir(f'{folder}/{cls}')
        
    # read all images to list
    for cls in classes:
        for name in image_names[cls]:
            img = cv2.imread(f'{folder}/{cls}/{name}', cv2.IMREAD_GRAYSCALE)
#             img = sitk.ReadImage(f'{folder}/{cls}/{name}', sitk.sitkUInt8)
#             mask_name = name.split('.')[0]+"_label_ground-truth_coco-panoptic.png"
            mask_name = name.split('.')[0]+"_label_ground-truth_semantic.png"
            mask = cv2.imread(f'{folder}/instance_mask/{mask_name}', cv2.IMREAD_GRAYSCALE)
#             mask = sitk.ReadImage(f'instance_mask/{mask_name}', sitk.sitkUInt8)
            images.append((name, img, cls, mask))
    return images

# Padding Images

In [ ]:
def padding(img, mask, x=512, y=512):
    img = sitk.GetArrayFromImage(img)
    mask = sitk.GetArrayFromImage(mask)
    
    pad_img = np.zeros((x,y))
    pad_mask = np.zeros((x,y))
    
    pad_img[:img.shape[0],:img.shape[1]] = img
    pad_mask[:mask.shape[0],:mask.shape[1]] = mask
    
#     pad_img = sitk.GetImageFromArray(pad_img)
#     pad_mask = sitk.GetImageFromArray(pad_mask)
    
    return pad_img, pad_mask

# CNN

# Detectron

In [3]:
import os
import json
import urllib.request

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.checkpoint import Checkpointer
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.data.datasets import register_coco_instances, load_coco_json

import numpy as np
import matplotlib.pyplot as plt

from pycocotools import mask

from segments.utils import export_dataset


In [4]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml'))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = 'model_final.pth'
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
predictor = DefaultPredictor(cfg)

In [5]:
class Model:
    def __init__(self, predictor):
        self.predictor = predictor

    def _convert_to_segments_format(self, image, outputs):
        # https://tech.amikelive.com/node-718/what-object-categories-labels-are-in-coco-dataset/
        segmentation_bitmap = np.zeros((image.shape[0], image.shape[1]), np.uint32)
        annotations = []
        counter = 1
        instances = outputs['instances']
        for i in range(len(instances.pred_classes)):
            category_id = int(instances.pred_classes[i])
            instance_id = counter
            mask = instances.pred_masks[i].cpu()
            segmentation_bitmap[mask] = instance_id
            annotations.append({'id': instance_id, 'category_id': category_id})
            counter += 1
        return segmentation_bitmap, annotations

    def __call__(self, image):
        image = np.array(image)
        outputs = self.predictor(image)
        label, label_data = self._convert_to_segments_format(image, outputs)

        return label, label_data

In [6]:
 model = Model(predictor)

In [7]:
images = read_images()
imgs = []
masks = []
count = 0

for name, img, cls, mask in images:
    imgs.append(img)
    masks.append(mask)
    count+=1
#     if count == 300:
#         train = (imgs, masks)
#         imgs = []
#         masks = []
    if count == 350:
        train = np.asarray([np.asarray(imgs), np.asarray(masks)])
        imgs = []
        masks = []
# valid = (imgs, masks)
test = np.asarray([np.asarray(imgs), np.asarray(masks)])

D:\Anaconda\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [8]:
test[0][0].shape

(537, 716)

In [9]:
segmentation_bitmap, annotations = model(test[0][0].reshape(537,716,1))

D:\Anaconda\lib\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [13]:
cv2.imwrite("img.png",segmentation_bitmap.astype('uint8'))

True

In [ ]:
plt.imshow(segmentation_bitmap.astype('uint8'))

In [ ]:
annotations

In [12]:
segmentation_bitmap.astype('uint8')

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)